In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
r = requests.get("https://www.nyrb.com/collections/classics")

In [5]:
html = r.text

In [7]:
soup = BeautifulSoup(html, "html5lib")

In [10]:
product_list = soup.find_all("div", class_="product")

In [19]:
def parse_product(product):
    print(product)
    img = product.find("img")['src']
    return {
        "img": img,
    }

In [20]:
parse_product(product_list[0])

<div class="product span2 adaptive-grid" data-pubdate="2019-02-19" id="product-negrophobia">
	
  
	<script>
		var saleAmount = Math.round(100 - 1495 / 0 * 100);
		if (saleAmount <= '20') $('#sale-negrophobia').remove();
	</script>

	<div class="productholder">
	  <div class="image">
		<a href="/collections/classics/products/negrophobia">
		  <img alt="Negrophobia" class="loop-image" src="//cdn.shopify.com/s/files/1/0726/9203/products/james.NEW_large.jpg?v=1532102917"/>
		</a>
	  </div>

	  <div class="details ploop">
		<a class="clearfix" href="/collections/classics/products/negrophobia">
		  <h4 class="title">Negrophobia</h4>
		  
		  <span class="author"><!-- sortauthorhandle: james collection: all --><!-- sortauthorhandle: james collection: ongiri --><!-- sortauthorhandle: james collection: james --><!-- sortauthorhandle: james collection: classics --></span>
			
            
        </a>
	  </div>
	</div>
</div>


{'img': '//cdn.shopify.com/s/files/1/0726/9203/products/james.NEW_large.jpg?v=1532102917'}

In [17]:
parsed_list = [parse_product(i) for i in product_list]

In [18]:
parsed_list

[{'img': '//cdn.shopify.com/s/files/1/0726/9203/products/james.NEW_large.jpg?v=1532102917'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/serge.1-1_3585e12f-64ea-4bf0-9595-c4ad7e579cbf_large.jpg?v=1528140914'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/multatuli.1_large.jpeg?v=1520874261'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/tergit.1_large.jpg?v=1520018643'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/viertel.final_large.jpg?v=1522100572'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/ozerov.2_large.jpg?v=1521046478'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/czapski.inhuman.land_NEW_large.jpg?v=1530219903'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/11_-_Lost_Time_large.jpeg?v=1519321987'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/Charles_Bovary_Country_Doctor_large.jpg?v=1531857991'},
 {'img': '//cdn.shopify.com/s/files/1/0726/9203/products/origo_war_large.jpg?v=1